In [1]:
import json
import random
import sys
import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.agents.context_qa_correction_agent import ContextQACorrectionAgent
from src.utils.data_generation import split_dataset, concurrent_data_postprocessing
random.seed(2)

In [2]:
PROC_NUM = 5
ERROR_DETECTION_ONLY = False
SPLIT = "train"

In [3]:
with open(f"../../../dataset/raw_model_responses/train/train_rajpurkar_squad.json", "r") as f:
    json_data = json.load(f)

In [4]:
json_data = json_data[:20]

In [5]:
data_chunks = split_dataset(json_data, PROC_NUM)

In [6]:
agent = ContextQACorrectionAgent(error_detection_only=ERROR_DETECTION_ONLY)

In [7]:
def extract_args(item: dict) -> list[str]:
    return {
        "input": item["input"],
        "question": item["additional_info"]["question"],
        "answer": item["additional_info"]["answer"],
        "context": item["additional_info"]["context"],
        "responses": item["response"],
    }

In [8]:
data = await concurrent_data_postprocessing(agent, data_chunks, extract_args)

2025-08-02 19:26:55.881 | INFO     | src.agents.context_qa_correction_agent:check_for_errors:103 - Errors: [[], [], [], [], [], [], [], [], [], []]
2025-08-02 19:26:55.882 | INFO     | src.agents.context_qa_correction_agent:check_for_errors:104 - Wrong responses: 0
2025-08-02 19:26:55.914 | INFO     | src.agents.context_qa_correction_agent:check_for_errors:103 - Errors: [[{'description': 'The response is incomplete. According to the context, elements of the fundamental group are represented by loops.', 'location': 'Elements of the fundamental group.', 'correction': 'ADD: Elements of the fundamental group are represented by loops.'}], [{'description': 'The response is incomplete. According to the context, elements of the fundamental group are represented by loops.', 'location': 'Elements of the fundamental group.', 'correction': 'ADD: Elements of the fundamental group are represented by loops.'}], [], [{'description': 'The response is incomplete. According to the context, elements of th

In [9]:
len(data)

20

In [10]:
with open(f"../../../dataset/processed_data/{SPLIT}/rajpurkar_squad_processed.json", "w") as f:
    json.dump(data, f, indent=4)